In [ ]:
import os
import platform
if platform.system() == "Windows":
    try:
        import requests
        from parsel import Selector
        from fake_headers import Headers
    except ImportError:
        os.system('python -m pip install requests')
        os.system('python -m pip install parsel')
        os.system('python -m pip install fake_headers')
else:
    try:
        import requests
        from parsel import Selector
        from fake_headers import Headers
    except ImportError:
        os.system('python3 -m pip install requests')
        os.system('python3 -m pip install parsel')
        os.system('python3 -m pip install fake_headers')


import requests
from parsel import Selector
import csv
import os
import time
import random
from fake_headers import Headers

def getdata(link):
    if link not in alreadyscrapped:
        req = requests.get(link,headers=Headers().generate())
        time.sleep(random.randint(3,8))
        response = Selector(text=req.text)

        title = response.xpath('.//h1/text()').extract_first()
        try:
            image_link = response.xpath('.//*[@class="scholarship-card"]/div/@style').extract_first().replace("background-image: url('","").replace("')","")
        except:
            image_link = ''
        tution_structure = '\n'.join(response.xpath('.//[@class="icon-dollor"]/following-sibling::text() | .//[@class="icon-dollor"]/following-sibling::span/text()').extract())
        university_name = response.xpath('.//[@class="icon-place"]/following-sibling::text() | .//[@class="icon-place"]/following-sibling::span/text()').extract_first()
        degree = response.xpath('.//*[@class="icon-Bachelor2"]/following-sibling::text()').extract_first()
        subject = response.xpath('.//[@class="icon-book"]/following-sibling::text() | .//[@class="icon-book"]/following-sibling::span/text()').extract_first()
        eligiblity = '\n'.join(response.xpath('.//[@class="icon-world"]/following-sibling::text() | .//[@class="icon-world"]/following-sibling::span/text()').extract())
        country = response.xpath('.//*[@class="icon-map"]/following-sibling::text()').extract_first()
        apply_date = response.xpath('.//*[@class="icon-calendar"]/following-sibling::text()').extract_first()
        try:
            scholarship_details = '\n'.join([i.strip() for i in response.xpath('.//*[@class="scholarship-details"]//text()').extract() if '{' not in i and 'View' not in i and i.strip() and 'Share' not in i])
        except:
            scholarship_details = ''


        with open("scholarshipads.csv","a",newline="",encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow([link,title,image_link,tution_structure,university_name,degree,subject,eligiblity,country,apply_date,scholarship_details])
            print([link,title,image_link,tution_structure,university_name,degree,subject,eligiblity,country,apply_date,scholarship_details])

    else:
        print("Exists ...")

def scrape_country(link):
    req = requests.get(link,headers=Headers().generate())
    time.sleep(random.randint(3,8))
    response = Selector(text=req.text)
    links = response.xpath('.//h2/a/@href[contains(.,"http")]').extract()
    for link in links:
        print("link: "+str(link))
        getdata(link)


    nextlink = response.xpath('.//*[@class="next "]/@href').extract_first()
    if nextlink:
        print("Nextlink :"+str(nextlink))
        scrape_country(nextlink)


#if _name_ == '_main_':

    if "scholarshipads.csv" not in os.listdir(os.getcwd()):
        with open("scholarshipads.csv","a",newline="",encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(['link','title','image_link','tution_structure','university_name','degree','subject','eligiblity','country','apply_date','scholarship details'])

    alreadyscrapped = []
    with open("scholarshipads.csv","r") as r:
        reader = csv.reader(r)
        for line in reader:
            alreadyscrapped.append(line[0])


    print(alreadyscrapped)
    url = "https://www.scholarshipsads.com/awarding-countries/"
    req = requests.get(url,headers=Headers().generate())
    time.sleep(random.randint(3,8))

    response = Selector(text=req.text)
    country_links = list(set(response.xpath('.//*[@class="card-body"]/ul/li/a/@href').extract()))

    for country_link in country_links:
        scrape_country(country_link)